In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
# # Defina as classes
classes = ['apple', 'banana']

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'UNDB\Cadeira-de-IA\PBL2-Aplicações de Visão Computacional na Rede Mateus\Dataset\Training',  
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Crie um novo ImageDataGenerator para o conjunto de validação
val_datagen = ImageDataGenerator(rescale=1./255)

# Gere os dados de validação
validation_generator = val_datagen.flow_from_directory(
    'UNDB\Cadeira-de-IA\PBL2-Aplicações de Visão Computacional na Rede Mateus\Dataset\Validation',  
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Calcule o número de etapas de validação
num_validation_samples = len(validation_generator.filenames)
validation_steps = num_validation_samples // validation_generator.batch_size

Found 1202 images belonging to 2 classes.
Found 375 images belonging to 2 classes.


In [3]:
# Carregar a arquitetura MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Adicionar uma camada de GlobalAveragePooling2D
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Adicionar uma camada densa com o número de classes como unidades e ativação softmax
predictions = Dense(len(classes), activation='softmax')(x)

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 [==============================] - 0s 0us/step


In [4]:
# Compilar o modelo
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Treinar o modelo
model.fit(train_generator,
          validation_data=validation_generator,
          steps_per_epoch=len(train_generator),
          validation_steps=validation_steps,
          epochs=10)

Epoch 1/10
38/38 [==============================] - 319s 8s/step - loss: 0.1038 - accuracy: 0.9617 - val_loss: 14.4476 - val_accuracy: 0.6932
Epoch 2/10
38/38 [==============================] - 7s 195ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 10.2857 - val_accuracy: 0.7898
Epoch 3/10
38/38 [==============================] - 5s 141ms/step - loss: 4.7488e-04 - accuracy: 1.0000 - val_loss: 8.4304 - val_accuracy: 0.8324
Epoch 4/10
38/38 [==============================] - 6s 150ms/step - loss: 8.9233e-05 - accuracy: 1.0000 - val_loss: 7.0357 - val_accuracy: 0.8352
Epoch 5/10
38/38 [==============================] - 6s 164ms/step - loss: 2.6882e-04 - accuracy: 1.0000 - val_loss: 5.0992 - val_accuracy: 0.8608
Epoch 6/10
38/38 [==============================] - 5s 142ms/step - loss: 2.5118e-04 - accuracy: 1.0000 - val_loss: 5.1170 - val_accuracy: 0.8551
Epoch 7/10
38/38 [==============================] - 6s 147ms/step - loss: 0.0012 - accuracy: 0.9992 - val_loss: 3.1148 - val_accura

In [8]:
# Carregar a imagem
img_path = 'UNDB\Cadeira-de-IA\PBL2-Aplicações de Visão Computacional na Rede Mateus\Dataset\Test\teste-21.jpg'
img = image.load_img(img_path, target_size=(64, 64))

# Converter a imagem para um array numpy e normalizá-la
img_array = image.img_to_array(img) / 255.

# Expandir as dimensões da imagem para que ela possa ser passada para o modelo
img_array = np.expand_dims(img_array, axis=0)

# Fazer a previsão
predictions = model.predict(img_array)

# Obter a classe prevista
predicted_class = classes[np.argmax(predictions)]

print(f"A imagem foi classificada como '{predicted_class}'.")

1/1 [==============================] - 0s 21ms/step
A imagem foi classificada como 'apple'.
